This notebook will be mainly used for the capstone project

In [121]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<h1>Below is my submission for Week 3: Segmenting and Clustering Neighborhoods in Toronto, Part 1</h1>

A quick Google search has revealed that the table I need is on the link below:

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [122]:
#read the tables on the page, checking the appearance of each
tbls=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
i=0
for t in tbls:
    i=i+1
    print('Table '+str(i)+': ',t.head())

Table 1:                                                     0                1  \
0                                    M1ANot assigned  M2ANot assigned   
1                    M1BScarborough(Malvern / Rouge)  M2BNot assigned   
2  M1CScarborough(Rouge Hill / Port Union / Highl...  M2CNot assigned   
3  M1EScarborough(Guildwood / Morningside / West ...  M2ENot assigned   
4                             M1GScarborough(Woburn)  M2GNot assigned   

                                                2  \
0                        M3ANorth York(Parkwoods)   
1                   M3BNorth York(Don Mills)North   
2  M3CNorth York(Don Mills)South(Flemingdon Park)   
3                                 M3ENot assigned   
4                                 M3GNot assigned   

                                                3  \
0                 M4ANorth York(Victoria Village)   
1  M4BEast York(Parkview Hill / Woodbine Gardens)   
2                  M4CEast York(Woodbine Heights)   
3                   

It's clear that Table 1, tbls[0], is the one I want. Focus on this one from now on.

In [123]:
t=tbls[0]
table_contents=[]

In [124]:
for rw in range(t.shape[0]):
    for cl in range(t.shape[1]):
        cell = {}
        txt=str(t.iloc[rw,cl])
        if txt[3:]=='Not assigned':
            pass
        else:
            cell['PostalCode']=txt[:3]
            the_rest=txt[3:]
            cell['Borough']=the_rest.split('(')[0]
            cell['Neighborhood']=the_rest.split('(')[1].split(')')[0].replace(' /',',')
            table_contents.append(cell)

In [125]:
d=pd.DataFrame(table_contents)
#replace a few Borough names as listed in 'Hints for scraping Notebook'
d['Borough']=d['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [126]:
#verify that the table has the desired shape
d

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [127]:
d.shape

(103, 3)

<h1>Below is my submission for Week 3: Segmenting and Clustering Neighborhoods in Toronto, Part 2</h1>

In [128]:
#using the url provided for geospatial coordinates, download the csv
url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
gsc=pd.read_csv(url)

In [129]:
#inspect the csv
gsc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [130]:
#Postal Code has a space. Align with the spelling in d
gsc=gsc.rename(columns={'Postal Code':'PostalCode'})

In [131]:
#do a merge on PostalCode
d=d.merge(gsc,how='left',on='PostalCode')

In [132]:
#check for postcodes where the merge might not have worked
d[pd.isnull(d.Latitude)],d[pd.isnull(d.Longitude)]

(Empty DataFrame
 Columns: [PostalCode, Borough, Neighborhood, Latitude, Longitude]
 Index: [],
 Empty DataFrame
 Columns: [PostalCode, Borough, Neighborhood, Latitude, Longitude]
 Index: [])

In [133]:
d

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<h1>Below is my submission for Week 3: Segmenting and Clustering Neighborhoods in Toronto, Part 3</h1>

I have decided to work with all boroughs, not only ones that contain the word 'Toronto'

In [134]:
import folium

The link below shows that the coordinates for Toronto are 43.651070, -79.347015
https://www.latlong.net/place/toronto-on-canada-27230.html

In [135]:
#assign the database previously created to neighborhoods
neighborhoods=d

#create a map of Toronto
t_lat,t_long=43.651070,-79.347015
map_toronto=folium.Map(location=[t_lat,t_long],zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

In [136]:
map_toronto

In [137]:
#the uneven distribution suggests the use of density based clustering
from sklearn.cluster import DBSCAN

In [139]:
#Plot the map of Toronto again, this time making the color vary with the cluster value. To do this, add a color map.
import branca.colormap as cm

In [140]:
X=neighborhoods[['Latitude','Longitude']]
epsilon=0.025
minimumSamples=4
db=DBSCAN(eps=epsilon,min_samples=minimumSamples).fit(X)
neighborhoods['clstr']=db.labels_
#check how many clusters we get
w=set(neighborhoods.clstr)

#colormap=cm.LinearColormap(colors=['red','lightblue'],index=[min(w),max(w)],vmin=min(w),vmax=max(w))
colormap=cm.LinearColormap(colors=['red','blue'],index=[min(w),max(w)])
map_toronto=folium.Map(location=[t_lat,t_long],zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, cls in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood'], list(neighborhoods['clstr'])):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=colormap(cls),
        fill=True,
        fill_color=colormap(cls),
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto